In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy

# Benchmark dynamic circuits with cut Bell pairs

*Bruuche-Schätzung: 22 Sekunde op enem Heron r2 Prozessor (OPJEPASS: Dat es nor en Schätzung. Ding Laafzick kann angers sin.)*
## Background
Quantehardware es normalerwies op lokale Interaktione beschränk, ävver ville Algorithme bruuche et, dat mer Qubits verschränke, die wigg usseneen ligge oder [Qubits op verschiedene Prozessore](#references). Dynamische Schaltkreise - dat sin Schaltkreise met Messunge un Feedforward widder om Laafe - sorge doför, dat mer die Begrenzunge ömgohn künne, indämm mer Echt-Zick-Kommunikation bruuche, öm effektiv nit-lokale Quanteoperatione ze implementiere. Domet künne Messunge us enem Deel vum Schaltkreis (oder enem QPU) bedingt Gates op enem andere triggere, su dat mer Verschränkung üvver groß Distanze teleportiere künne. Dat es d Basis vun **lokale Operatione un klassische Kommunikation (LOCC)** Scheme, wo mer verschränkte Ressource-Zustäng (Bell-Paare) verbruuche un Messresultate klassisch kommuniziere, öm wigg usseneen liegend Qubits ze verbinge.

En vielversprechend Nützung vun LOCC es et, virtuelli Lang-Strecke-CNOT-Gates durch Teleportation ze realisiere, wie em [long-range entanglement tutorial](/tutorials/long-range-entanglement) jezeich. Anstatt enem direkte Lang-Strecke-CNOT (wat d Hardware-Konnektivität velleisch nit erlaub), erstelle mer Bell-Paare un föhre en teleportationsbasiert Gate-Implementierung durch. Allerdings hängk d Fidelität vun son Operatione vun d Hardwareeigenschaften av. Qubit-Dekohärenz widder de nötije Verzögerung (während mer op Messresultate waade) un klassische Kommunikationslatenz künne d verschränkte Zostand verschlechtere. Außerdem sin Fähler bei Messunge widder om Laafe schwieriger ze korrigiere als Fähler bei d Endmessunge, weil se sich durch de bedingde Gates op d Räss vum Schaltkreis usbreede.

Em [Referenz-Experiment](#references) stelle d Autore e Bell-Paar-Fidelitäts-Benchmark vör, öm eruuszefinge, welche Deile vun enem Gerät am beschte för LOCC-basiert Verschränkung geeignet sin. D Idee es et, ene kleine dynamische Schaltkreis op jeder Grupp vun vier verbundene Qubits em Prozessor ze laafe losse. Dä Vier-Qubit-Schaltkreis erstellt zorschtemol e Bell-Paar op zwei mittlere Qubits, un dann bruuch dat die als Ressource, öm de zwei Rand-Qubits durch LOCC ze verschränke. Konkret wäde Qubits 1 un 2 lokal en en ungeschnitte Bell-Paar präpariert (met enem Hadamard un CNOT), un dann verbruuch en Teleportationsroutine dat Bell-Paar, öm Qubits 0 un 3 ze verschränke. Qubits 1 un 2 wäde widder d Usführung vum Schaltkreis jemesse, un basierend op dänne Ergebnisse wäde Pauli-Korrektore (en X op Qubit 3 un Z op Qubit 0) aanjewendt. Qubits 0 un 3 blieve dann am Engk vum Schaltkreis en enem Bell-Zostand.

Öm d Qualität vun däm Eng-Bell-Paar ze quantifiziere, messe mer sing Stabilisatore: konkret d Parität en d $Z$-Basis ($Z_0Z_3$) un en d $X$-Basis ($X_0X_3$). För e perfek Bell-Paar sin beeds dä Erwartungswääte +1. En d Praxis weed Hardware-Rausche die Wääte reduziere. Doröm widderhohle mer d Schaltkreis zweimol för jedes Qubit-Paar: eene Schaltkreis miss Qubits 0 un 3 en d $Z$-Basis, un en andere miss se en d $X$-Basis. Us dänne Resultate krijje mer en Schätzung vun $\langle Z_0Z_3\rangle$ un $\langle X_0X_3\rangle$ för dat Paar vun Qubits. Mer bruuche d mittler quadratisch Afwichung (MSE) vun dänne Stabilisatore bezoore op d ideale Wäät (1) als einfach Metrik vun d Verschränkungsfidelität. En niedrigere MSE bedügg, dat de zwei Qubits ene Bell-Zostand nöher am Ideale erreicht han (hüherer Fidelität), während en hüherer MSE mieh Fähler anzeich. Indämm mer dat Experiment üvver dat Gerät scanne, künne mer d Mess-un-Feedforward-Fähigkeit vun verschiedene Qubit-Gruppe benchmarke un de beschte Paare vun Qubits för LOCC-Operatione identifiziere.

Dat Tutorial demonstriert dat Experiment op enem IBM Quantum&reg; Gerät, öm ze zeije, wie dynamische Schaltkreise jebruch wäde künne, öm Verschränkung zwesche wigg usseneen liegend Qubits ze erzeuche un ze evaluiere. Mer wäde all Vier-Qubit-lineare Kette op däm Gerät kartiere, d Teleportationsschaltkreis op jeder laafe losse, un dann d Verdeiling vun d MSE-Wääte visualisiere. Dä End-zu-End-Prozedur zeich, wie mer Qiskit Runtime un dynamische Schaltkreisfunktione nutze künne, öm hardware-bewusste Entscheidunge för dat Schnigge vun Schaltkreise oder dat Verdeilinge vun Quantealgorithme üvver e modulare System ze treffe.
## Requirements
Bevör De met däm Tutorial aanfängks, stell secher, dat De Foljendes installiert häss:

* Qiskit SDK v2.0 oder neuer, met [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization) Ungerstötzung
* Qiskit Runtime v0.40 oder neuer (`pip install qiskit-ibm-runtime`)
## Setup

In [ ]:
from qiskit import QuantumCircuit

from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit.transpiler import generate_preset_pass_manager

import numpy as np
import matplotlib.pyplot as plt


def create_bell_stab(initial_layouts):
    """
    Create a circuit for a 1D chain of qubits (number of qubits must be a multiple of 4),
    where a middle Bell pair is consumed to create a Bell at the edge.
    Takes as input a list of lists, where each element of the list is a
    1D chain of physical qubits that is used as the initial_layout for the transpiled circuit.
    Returns a list of length-2 tuples, each tuple contains a circuit to measure the ZZ stabilizer and
    a circuit to measure the XX stabilizer of the edge Bell state.
    """
    bell_circuits = []
    for (
        initial_layout
    ) in initial_layouts:  # Iterate over chains of physical qubits
        assert (
            len(initial_layout) % 4 == 0
        ), f"The length of the chain must be a multiple of 4, len(inital_layout)={len(initial_layout)}"
        num_pairs = len(initial_layout) // 4

        bell_parallel = QuantumCircuit(4 * num_pairs, 4 * num_pairs)

        for pair_idx in range(num_pairs):
            (q0, q1, q2, q3) = (
                pair_idx * 4,
                pair_idx * 4 + 1,
                pair_idx * 4 + 2,
                pair_idx * 4 + 3,
            )
            (c0, c1) = pair_idx * 4, pair_idx * 4 + 3  # edge qubits
            (ca0, ca1) = pair_idx * 4 + 1, pair_idx * 4 + 2  # middle qubits

            bell_parallel.h(q0)
            bell_parallel.h(q1)
            bell_parallel.cx(q1, q2)
            bell_parallel.cx(q0, q1)
            bell_parallel.cx(q2, q3)
            bell_parallel.h(q2)

        # add barrier BEFORE measurements and add id in conditional
        bell_parallel.barrier()
        for pair_idx in range(num_pairs):
            (q0, q1, q2, q3) = (
                pair_idx * 4,
                pair_idx * 4 + 1,
                pair_idx * 4 + 2,
                pair_idx * 4 + 3,
            )
            (ca0, ca1) = pair_idx * 4 + 1, pair_idx * 4 + 2  # middle qubits

            bell_parallel.measure(q1, ca0)
            bell_parallel.measure(q2, ca1)
        # bell_parallel.barrier() #remove barrier after measurement

        for pair_idx in range(num_pairs):
            (q0, q1, q2, q3) = (
                pair_idx * 4,
                pair_idx * 4 + 1,
                pair_idx * 4 + 2,
                pair_idx * 4 + 3,
            )
            (ca0, ca1) = pair_idx * 4 + 1, pair_idx * 4 + 2  # middle qubits
            with bell_parallel.if_test((ca0, 1)):
                bell_parallel.x(q3)
            with bell_parallel.if_test((ca1, 1)):
                bell_parallel.z(q0)
                bell_parallel.id(q0)  # add id here for correct alignment

        bell_zz = bell_parallel.copy()
        bell_zz.barrier()
        bell_xx = bell_parallel.copy()
        bell_xx.barrier()
        for pair_idx in range(num_pairs):
            (q0, q1, q2, q3) = (
                pair_idx * 4,
                pair_idx * 4 + 1,
                pair_idx * 4 + 2,
                pair_idx * 4 + 3,
            )
            bell_xx.h(q0)
            bell_xx.h(q3)
        bell_xx.barrier()
        for pair_idx in range(num_pairs):
            (q0, q1, q2, q3) = (
                pair_idx * 4,
                pair_idx * 4 + 1,
                pair_idx * 4 + 2,
                pair_idx * 4 + 3,
            )
            (c0, c1) = pair_idx * 4, pair_idx * 4 + 3  # edge qubits

            bell_zz.measure(q0, c0)
            bell_zz.measure(q3, c1)

            bell_xx.measure(q0, c0)
            bell_xx.measure(q3, c1)

        bell_circuits.append(bell_zz)
        bell_circuits.append(bell_xx)

    return bell_circuits


def get_mse(result, initial_layouts):
    """
    given a result object and the initial layouts, returns a dict of layouts and their mse
    """
    layout_mse = {}
    for layout_idx, initial_layout in enumerate(initial_layouts):
        layout_mse[tuple(initial_layout)] = {}

        num_pairs = len(initial_layout) // 4

        counts_zz = result[2 * layout_idx].data.c.get_counts()
        total_shots = sum(counts_zz.values())

        # Get ZZ expectation value
        exp_zz_list = []
        for pair_idx in range(num_pairs):
            exp_zz = 0
            for bitstr, shots in counts_zz.items():
                bitstr = bitstr[::-1]  # reverse order to big endian
                b1, b0 = (
                    bitstr[pair_idx * 4],
                    bitstr[pair_idx * 4 + 3],
                )  # parse bitstring to get edge measurements for each 4-q chain
                z_val0 = 1 if b0 == "0" else -1
                z_val1 = 1 if b1 == "0" else -1
                exp_zz += z_val0 * z_val1 * shots
            exp_zz /= total_shots
            exp_zz_list.append(exp_zz)

        counts_xx = result[2 * layout_idx + 1].data.c.get_counts()
        total_shots = sum(counts_xx.values())

        # Get XX expectation value
        exp_xx_list = []
        for pair_idx in range(num_pairs):
            exp_xx = 0
            for bitstr, shots in counts_xx.items():
                bitstr = bitstr[::-1]  # reverse order to big endian
                b1, b0 = (
                    bitstr[pair_idx * 4],
                    bitstr[pair_idx * 4 + 3],
                )  # parse bitstring to get edge measurements for each 4-q chain
                x_val0 = 1 if b0 == "0" else -1
                x_val1 = 1 if b1 == "0" else -1
                exp_xx += x_val0 * x_val1 * shots
            exp_xx /= total_shots
            exp_xx_list.append(exp_xx)

        mse_list = [
            ((exp_zz - 1) ** 2 + (exp_xx - 1) ** 2) / 2
            for exp_zz, exp_xx in zip(exp_zz_list, exp_xx_list)
        ]

        print(f"layout {initial_layout}")
        for idx in range(num_pairs):
            layout_mse[tuple(initial_layout)][
                tuple(initial_layout[4 * idx : 4 * idx + 4])
            ] = mse_list[idx]
            print(
                f"qubits: {initial_layout[4*idx:4*idx+4]}, mse:, {round(mse_list[idx],4)}"
            )
            # print(f'exp_zz: {round(exp_zz_list[idx],4)}, exp_xx: {round(exp_xx_list[idx],4)}')
        print(" ")
    return layout_mse


def plot_mse_ecdfs(layouts_mse, combine_layouts=False):
    """
    Plot CDF of MSE data for multiple layouts. Optionally combine all data in a single CDF
    """

    if not combine_layouts:
        for initial_layout, layouts in layouts_mse.items():
            sorted_layouts = dict(
                sorted(layouts.items(), key=lambda item: item[1])
            )  # sort layouts by mse

            # get layouts and mses
            layout_list = list(sorted_layouts.keys())
            mse_list = np.asarray(list(sorted_layouts.values()))

            # convert to numpy
            x = np.array(mse_list)
            y = np.arange(1, len(x) + 1) / len(x)

            # Prepend (x[0], 0) to start CDF at zero
            x = np.insert(x, 0, x[0])
            y = np.insert(y, 0, 0)

            # Create the plot
            plt.plot(
                x,
                y,
                marker="x",
                linestyle="-",
                label=f"qubits: {initial_layout}",
            )

            # add qubits labels for the edge pairs
            for xi, yi, q in zip(x[1:], y[1:], layout_list):
                plt.annotate(
                    [q[0], q[3]],
                    (xi, yi),
                    textcoords="offset points",
                    xytext=(5, -10),
                    ha="left",
                    fontsize=8,
                )

    elif combine_layouts:
        all_layouts = {}
        all_initial_layout = []
        for (
            initial_layout,
            layouts,
        ) in layouts_mse.items():  # puts together all layout information
            all_layouts.update(layouts)
            all_initial_layout += initial_layout

        sorted_layouts = dict(
            sorted(all_layouts.items(), key=lambda item: item[1])
        )  # sort layouts by mse

        # get layouts and mses
        layout_list = list(sorted_layouts.keys())
        mse_list = np.asarray(list(sorted_layouts.values()))

        # convert to numpy
        x = np.array(mse_list)
        y = np.arange(1, len(x) + 1) / len(x)

        # Prepend (x[0], 0) to start CDF at zero
        x = np.insert(x, 0, x[0])
        y = np.insert(y, 0, 0)

        # Create the plot
        plt.plot(
            x,
            y,
            marker="x",
            linestyle="-",
            label=f"qubits: {sorted(list(set(all_initial_layout)))}",
        )

        # add qubit labels for the edge pairs
        for xi, yi, q in zip(x[1:], y[1:], layout_list):
            plt.annotate(
                [q[0], q[3]],
                (xi, yi),
                textcoords="offset points",
                xytext=(5, -10),
                ha="left",
                fontsize=8,
            )

    plt.xscale("log")
    plt.xlabel("Mean squared error of ⟨ZZ⟩ and ⟨XX⟩")
    plt.ylabel("Cumulative distribution function")
    plt.title("CDF for different initial layouts")
    plt.grid(alpha=0.3)
    plt.show()

## Step 1: Map classical inputs to a quantum problem
D erschte Schrett es et, en Satz vun Quanteschaltkreise ze erstelle, öm all kandidat Bell-Paar-Links ze benchmarke, die op d Topologie vum Gerät aanjepass sin. Mer duerschuche programmatesch d Gerät-Kopplungskaart för all linear verbundene Kette vun vier Qubits. Jede son Kette (durch Qubit-Indizes $[q0-q1-q2-q3]$ jezeichnet) dient als Testfall för d Verschränkungs-Swapping-Schaltkreis. Indämm mer all möjjlich Länge-4-Pade identifiziere, stelle mer maximali Deckung för möjjlich Gruppierunge vun Qubits secher, die dat Protokoll realisiere künnte.

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(operational=True)

Mer erzeuche die Kette, indämm mer en Hilfsfunktion bruuche, die en gierige Suche op däm Gerät-Graf durchföhrt. Se gitt "Streife" vun vier Vier-Qubit-Kette zeröck, die en 16-Qubit-Gruppe gebüngelt sin (dynamische Schaltkreise beschränke momentan d Jrüßte vum Messregister op `16` Qubits). Dat Büngele erlaub et oos, mihrere Vier-Qubit-Experimente parallel op verschiedene Deile vum Chip ze laafe losse un dat janze Gerät effizient ze bruuche. Jede 16-Qubit-Streife enthält vier disjunkte Kette, dat heiß, dat kei Qubit binnen dä Grupp widder jebruch weed. Zom Beispill künnt eene Streife us Kette $[0-1-2-3]$, $[4-5-6-7]$, $[8-9-10-11]$ un $[12-13-14-15]$ bestohn, all zesamme jepack. Jedes Qubit, dat nit en enem Streife enthalte wohr, weed en d `leftover` Variable zeröckjejevve.

In [79]:
from itertools import chain
from collections import defaultdict


def stripes16_from_backend(backend):
    """
    Creates stripes of 16 qubits, four non-overlapping  four-qubit chains, that cover as much of
    the coupling map as possible. Returns any unused qubits as leftovers.
    """
    # get the undirected adjacency list
    edges = backend.coupling_map.get_edges()
    graph = defaultdict(set)
    for u, v in edges:
        graph[u].add(v)
        graph[v].add(u)

    qubits = sorted(graph)  # all qubit indices that appear

    # greedy search for 4-long linear chains (blocks) ────────────
    used = set()  # qubits already placed in a block
    blocks = []  # each block is a four-qubit list

    for q in qubits:  # deterministic order for reproducibility
        if q in used:
            continue  # already consumed by earlier block

        # depth-first "straight" walk of length 3 without revisiting nodes
        def extend(path):
            if len(path) == 4:
                return path
            tip = path[-1]
            for nbr in sorted(graph[tip]):  # deterministic
                if nbr not in path and nbr not in used:
                    maybe = extend(path + [nbr])
                    if maybe:
                        return maybe
            return None

        block = extend([q])
        if block:  # found a 4-node path
            blocks.append(block)
            used.update(block)

    # bundle four four-qubit blocks into one 16-qubit stripe (max number of measurement compatible with if-else)
    stripes = [
        list(chain.from_iterable(blocks[i : i + 4]))
        for i in range(0, len(blocks) // 4 * 4, 4)  # full groups of four
    ]

    leftovers = set(qubits) - set(chain.from_iterable(stripes))
    return stripes, leftovers

In [80]:
initial_layouts, leftover = stripes16_from_backend(backend)

Als nächstes konstruiere mer d Schaltkreis för jede 16-Qubit-Streife. D Routine deit dat Foljende för jede Kette:

* E mittle Bell-Paar präpariere: E Hadamard op Qubit 1 un e CNOT vun Qubit 1 noh Qubit 2 aaweinde. Dat verschränk Qubits 1 un 2 (un erstellt ene $|\Phi^+\rangle = (|00\rangle + |11\rangle)/\sqrt{2}$ Bell-Zostand).
* D Rand-Qubits verschränke: E CNOT vun Qubit 0 noh Qubit 1 un e CNOT vun Qubit 2 noh Qubit 3 aaweinde. Dat verbingk de aanfänglich jedrennte Paare, su dat Qubits 0 un 3 noh d nächste Schrette verschränk wäde. E Hadamard op Qubit 2 weed och aanjewendt (dat, kombiniert met d frühere CNOTs, bild en Deel vun ener Bell-Messung op Qubits 1 un 2). Zo däm Ziggpunk sin Qubits 0 un 3 noch nit verschränk, ävver Qubits 1 un 2 sin met dänne en enem jrüßere Vier-Qubit-Zostand verschränk.
* Messunge widder om Laafe un Feedforward: Qubits 1 un 2 (de mittle Qubits) wäde en d Berechnungsbasis jemesse, wat zwei klassische Bits ergitt. Basierend op dänne Messresultate weinde mer bedingte Operatione aan: Wann d Qubit-1-Messung (dat nemme mer Bit $m_{12}$) 1 es, weinde mer en $X$-Gate op Qubit 3 aan; wann d Qubit-2-Messung ($m_{21}$) 1 es, weinde mer en $Z$-Gate op Qubit 0 aan. Die bedingte Gates (realisiert durch d Qiskit `if_test`/`if_else` Konstrukt) implementiere de standard Teleportationskorrektore. Se "maache rückjängjig" de zofallige Pauli-Flips, die durch dat Projiziere vun Qubits 1 un 2 passiere, un stelle secher, dat Qubits 0 un 3 en enem bekannte Bell-Zostand ennde, unafhängjig vun d Messresultate. Noh däm Schrett sollte Qubits 0 un 3 idealerwies em Bell-Zostand $|\Phi^+\rangle$ verschränk sin.
* Bell-Paar-Stabilisatore messe: Mer deile dann en zwei Versione vum Schaltkreis. En d erschte Version messe mer d $ZZ$ Stabilisator op Qubits 0 un 3. En d zweite Version messe mer d $XX$ Stabilisator op dänne Qubits.

För jede Vier-Qubit-initial-Layout gitt de ov jenannte Funktion zwei Schaltkreise zeröck (eene för $ZZ$, eene för $XX$ Stabilisatormessung). Am Engk vun däm Schrett han mer en Leß vun Schaltkreise, die jede Vier-Qubit-Kette op däm Gerät decke. Die Schaltkreise enthalde Messunge widder om Laafe un bedingte (if/else) Operatione, wat de zentrale Aanweisunge vum dynamische Schaltkreis sin.

In [63]:
circuits = create_bell_stab(initial_layouts)
circuits[-1].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/edc-cut-bell-pair-benchmarking/extracted-outputs/bd04755f-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/edc-cut-bell-pair-benchmarking/extracted-outputs/bd04755f-0.avif)

## Step 2: Optimize the problem for quantum hardware execution
Bevör mer unse Schaltkreise op echter Hardware usföhre, mösse mer se transpiliere, öm ze d physische Beschränkunge vum Gerät ze passe. Transpilierung weed d abstrakte Schaltkreis op de physische Qubits un Gate-Satz vum usjesöchte Gerät mappe. Weil mer alld spezifische physische Qubits för jede Kette usjesöch han (indämm mer en `initial_layout` för d Schaltkreisgenerator metjevve han), bruuche mer Transpiler `optimization_level=0` met däm feste Layout. Dat säät Qiskit, dat et kein Qubits widder zuweise oder schwere Optimierunge durchföhre soll, die d Schaltkreisstruktur ännere künnte. Mer wolle de Sequenz vun Operatione (besonders de bedingte Gates) jenau su behalde, wie aanjejevve.

In [ ]:
isa_circuits = []
for ind, init_layout in enumerate(initial_layouts):
    pm = generate_preset_pass_manager(
        optimization_level=0, backend=backend, initial_layout=init_layout
    )
    isa_circ = pm.run(circuits[ind * 2 : ind * 2 + 2])
    isa_circuits.extend(isa_circ)

In [65]:
isa_circuits[1].draw("mpl", fold=-1, idle_wires=False)

<Image src="../docs/images/tutorials/edc-cut-bell-pair-benchmarking/extracted-outputs/3ad620f7-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/edc-cut-bell-pair-benchmarking/extracted-outputs/3ad620f7-0.avif)

## Step 3: Execute using Qiskit primitives
Jetz künne mer dat Experiment op däm Quantegerät laafe losse. Mer bruuche Qiskit Runtime un sing Sampler-Primitiv, öm d Charge vun Schaltkreise effizient uszföhre.

In [ ]:
sampler = Sampler(mode=backend)
sampler.options.environment.job_tags = ["cut-bell-pair-test"]
job = sampler.run(isa_circuits)

## Step 4: Post-process and return result in the desired classical format
D letzte Schrett es et, de mittler quadratisch Afwichungsmetrik (MSE) för jede jetestete Qubit-Grupp ze berechne un de Resultate zesammezefasse. För jede Kette han mer jetz dat jemessene $\langle Z_0Z_3\rangle$ un $\langle X_0X_3\rangle$. Wann Qubits 0 un 3 perfek en enem $|\Phi^+\rangle$ Bell-Zostand verschränk wöre, wöre mer erwarte, dat beeds vun dänne +1 sin. Mer quantifiziere d Afwichung met d MSE:

$$\text{MSE} = \frac{( \langle Z_0Z_3\rangle - 1)^2 + (\langle X_0X_3\rangle - 1)^2}{2}.$$

Dä Wäät es 0 för e perfek Bell-Paar un steich, wie d verschränkte Zostand rauschiger weed (met zofallige Resultate, die en Erwartung öm 0 gevve, wörd d MSE sich 1 nöhere). D Code berechnet die MSE för jede Vier-Qubit-Grupp.

De Resultate zeije en groß Spannwigg vun Verschränkungsqualität üvver dat Gerät. Dat bestätich d Entdeckung vum Paper, dat et üvver en Jrößeordnung Variation en Bell-Zostandsfidelität gevve kann, abhängjig dervun, welche physische Qubits jebruch wäde. En praktische Begriffe bedügg dat, dat bestemmpte Regione oder Links em Chip vill besser dorin sin, Messunge widder om Laafe un Feedforward-Operatione durchzeföhre als anger. Faktore wie Qubit-Auslesefähler, Qubit-Lävensdauer un Überspreche drage wohrscheinlich zo dänne Ongerscheede bei. Zom Beispill, wann en Kette e besonders rauschig Auslesequbit enthält, künnt d Messung widder om Laafe onzoverlässig sin, wat zo ener schlechte Fidelität för dat verschränkte Paar föhrt (hohe MSE).

In [71]:
layouts_mse = get_mse(job.result(), initial_layouts)

layout [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
qubits: [0, 1, 2, 3], mse:, 0.0312
qubits: [4, 5, 6, 7], mse:, 0.0491
qubits: [8, 9, 10, 11], mse:, 0.0711
qubits: [12, 13, 14, 15], mse:, 0.0436
 
layout [16, 23, 22, 21, 17, 27, 26, 25, 18, 31, 30, 29, 19, 35, 34, 33]
qubits: [16, 23, 22, 21], mse:, 0.0197
qubits: [17, 27, 26, 25], mse:, 0.113
qubits: [18, 31, 30, 29], mse:, 0.0287
qubits: [19, 35, 34, 33], mse:, 0.0433
 
layout [36, 41, 42, 43, 37, 45, 46, 47, 38, 49, 50, 51, 39, 53, 54, 55]
qubits: [36, 41, 42, 43], mse:, 0.1645
qubits: [37, 45, 46, 47], mse:, 0.0409
qubits: [38, 49, 50, 51], mse:, 0.0519
qubits: [39, 53, 54, 55], mse:, 0.0829
 
layout [56, 63, 62, 61, 57, 67, 66, 65, 58, 71, 70, 69, 59, 75, 74, 73]
qubits: [56, 63, 62, 61], mse:, 0.8663
qubits: [57, 67, 66, 65], mse:, 0.0375
qubits: [58, 71, 70, 69], mse:, 0.0664
qubits: [59, 75, 74, 73], mse:, 0.0291
 
layout [76, 81, 82, 83, 77, 85, 86, 87, 78, 89, 90, 91, 79, 93, 94, 95]
qubits: [76, 81, 82, 83], mse

Finally, we visualize the overall performance by plotting the cumulative distribution function (CDF) of the MSE values for all chains. The CDF plot shows the MSE threshold on the x-axis, and the fraction of qubit pairs that have at most that MSE on the y-axis. This curve starts at zero and approaches one as the threshold grows to encompass all data points. A steep rise near a low MSE would indicate that many pairs are high-fidelity; a slow rise means that many pairs have larger errors. We annotate the CDF with the identities of the best pairs. In the plot, each point in the CDF corresponds to one four-qubit chain's MSE, and we label the point with the pair of qubit indices $[q0, q3]$ that were entangled in that experiment. This makes it easy to spot which physical qubit pairs are the top performers (the far-left points on the CDF).

In [68]:
plot_mse_ecdfs(layouts_mse, combine_layouts=True)

<Image src="../docs/images/tutorials/edc-cut-bell-pair-benchmarking/extracted-outputs/678ddac9-0.avif" alt="Output of the previous code cell" />

Zum Schloss visualisiere mer d Jesampperformanz, indämm mer de kumulative Verdeilingsfunktion (CDF) vun d MSE-Wääte för all Kette plotte. Dat CDF-Diagramm zeich d MSE-Schwelle op d x-Achs un d Fraktion vun Qubit-Paare, die höchstens die MSE han, op d y-Achs. Die Kurve fängk bei null aan un nähert sich eins, wie d Schwelle wääs, öm all Datepunkte ze omfatte. En steile Aansteech bei ener niedrige MSE wörd anzeije, dat ville Paare en hoher Fidelität han; en langsamer Aansteech bedügg, dat ville Paare jrüßere Fähler han. Mer annotiere dat CDF met d Identitäte vun de beschte Paare. Em Diagramm entsprich jede Punk em CDF enem Vier-Qubit-Kette sing MSE, un mer zeichne d Punk met däm Paar vun Qubit-Indizes $[q0, q3]$, die en däm Experiment verschränk wohre. Dat maach et einfach, erauszefinge, welche physische Qubit-Paare de beste Leistung han (de Punkte ganz links op däm CDF).